# Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
from pickle import load

from datetime import datetime
from datetime import timedelta

# Constants
Tweakable Constant

In [ ]:
#DATA_FILE = 'DATA2wFRED.csv'
#DATA_FILE = 'DATA2.csv'
DATA_FILE = 'Back_Trading_Data.csv'

DATE_FROM = '2022-07-01'
DATE_TO = '2022-09-21'
DATE_FORMAT = '%Y-%m-%d'

In [ ]:
COST_BUY = 5
COST_HOLD = 0.2

CAPITAL = 10000

In [ ]:
#BACKWARD_LOOKING_NUM = 3
BACKWARD_LOOKING_NUM = 30

# Percentage
TREND_FLAT_THREHOLD = 0.05

# Trading fee = 5 USD
FEE_TRADING = 5

In [ ]:
FOURIER_COMPONENTS =[3,6,9,10,11,12]

In [ ]:
MODEL_FILE = 'Experiment_12_WGAN-GP.h5'
X_SCALE_PKL = 'X_scaler.pkl'
Y_SCALE_PKL = 'y_scaler.pkl'

# Load Data

In [ ]:
df = pd.read_csv(DATA_FILE, parse_dates=['Date'])
df.sort_values(by="Date", inplace=True)

In [ ]:
df.head()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,GBPUSD,GOLD,SILVER,COPPER,PLATINUM,PALLADIUM,OIL_CRUDE,OIL_BRENT,NATURALGAS,VIX
10,2012-01-06,20.327,20.452,20.019,20.035,520980376,1.877,27.720,14.288,8.993,...,1.5363,1624.00,28.5275,7373.00,1445.00,613.25,83.23,98.81,2.2400,26.66
52,2012-01-08,21.997,22.014,21.536,21.672,384498968,1.750,20.880,15.832,7.786,...,1.5536,1599.58,27.4250,7418.25,1398.40,582.81,88.91,105.83,3.2000,18.96
94,2012-01-10,23.970,24.170,23.446,23.550,543583684,1.944,21.990,19.063,8.007,...,1.6130,1775.43,34.6600,8300.75,1679.20,644.85,92.48,112.14,3.1900,16.32
115,2012-01-11,21.365,21.536,21.220,21.305,361447688,1.950,21.210,17.206,11.099,...,1.6130,1714.20,32.2575,7825.00,1571.03,612.45,87.09,108.82,3.5000,16.69
31,2012-02-07,20.883,21.195,20.843,21.161,400091104,2.027,30.771,14.526,9.693,...,1.5691,1597.01,27.4925,7634.75,1455.78,575.25,83.75,96.59,2.7341,16.80


In [ ]:
df.tail()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,GBPUSD,GOLD,SILVER,COPPER,PLATINUM,PALLADIUM,OIL_CRUDE,OIL_BRENT,NATURALGAS,VIX
2432,2022-12-04,165.28,169.90,165.22,168.61,201458,332.21,214.97,129.00,345.97,...,1.30025,1966.09,25.412,4.7185,973.60,2348.31,101.09,104.83,6.778,24.71
2451,2022-12-05,145.11,146.21,138.82,143.90,354955,248.08,194.21,113.88,177.98,...,1.22052,1821.63,20.825,4.1105,958.10,1953.35,105.51,107.18,7.789,30.51
2490,2022-12-07,143.71,148.47,143.65,145.87,115834,232.17,163.25,114.77,174.45,...,1.18783,1725.48,18.908,3.2525,846.86,2033.60,92.10,97.44,6.144,27.76
2513,2022-12-08,169.39,172.18,168.73,172.10,83724,300.02,180.50,122.92,249.42,...,1.21397,1802.86,20.826,3.6880,972.98,2229.06,91.37,97.19,8.755,21.20
2533,2022-12-09,158.21,164.29,158.21,164.20,111706,304.74,169.26,112.54,236.80,...,1.16899,1725.45,19.853,3.6205,910.46,2269.45,87.71,93.69,8.455,23.10


# Fourier Transform Data

In [ ]:
def get_technical_indicators(data):
    # Create 7 and 21 days Moving Average
    data['MA7'] = data.iloc[:,4].rolling(window=7).mean()
    data['MA21'] = data.iloc[:,4].rolling(window=21).mean()

    # Create MACD
    data['MACD'] = data.iloc[:,4].ewm(span=26).mean() - data.iloc[:,1].ewm(span=12,adjust=False).mean()

    # Create Bollinger Bands
    data['20SD'] = data.iloc[:, 4].rolling(20).std()
    data['upper_band'] = data['MA21'] + (data['20SD'] * 2)
    data['lower_band'] = data['MA21'] - (data['20SD'] * 2)

    # Create Exponential moving average
    data['EMA'] = data.iloc[:,4].ewm(com=0.5).mean()

    # Create LogMomentum
    data['logmomentum'] = np.log(data.iloc[:,4] - 1)

    return data

In [ ]:
def get_fourier_transfer(dataset, components):
    # Get the columns for doing fourier
    data_FT = dataset[['Date', 'Close']]

    close_fft = np.fft.fft(np.asarray(data_FT['Close'].tolist()))
    fft_df = pd.DataFrame({'fft': close_fft})
    fft_df['absolute'] = fft_df['fft'].apply(lambda x: np.abs(x))
    fft_df['angle'] = fft_df['fft'].apply(lambda x: np.angle(x))

    fft_list = np.asarray(fft_df['fft'].tolist())
    fft_com_df = pd.DataFrame()
    for num_ in components:
        fft_list_m10 = np.copy(fft_list);
        fft_list_m10[num_:-num_] = 0
        fft_ = np.fft.ifft(fft_list_m10)
        fft_com = pd.DataFrame({'fft': fft_})
        fft_com['absolute of ' + str(num_) + ' comp'] = fft_com['fft'].apply(lambda x: np.abs(x))
        fft_com['angle of ' + str(num_) + ' comp'] = fft_com['fft'].apply(lambda x: np.angle(x))
        fft_com = fft_com.drop(columns='fft')
        fft_com_df = pd.concat([fft_com_df, fft_com], axis=1)

    return fft_com_df

In [ ]:
T_df = get_technical_indicators(df)

In [ ]:
T_df.head()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,NATURALGAS,VIX,MA7,MA21,MACD,20SD,upper_band,lower_band,EMA,logmomentum
10,2012-01-06,20.327,20.452,20.019,20.035,520980376,1.877,27.720,14.288,8.993,...,2.2400,26.66,NaN,NaN,-0.292000,NaN,NaN,NaN,20.035000,2.946279
52,2012-01-08,21.997,22.014,21.536,21.672,384498968,1.750,20.880,15.832,7.786,...,3.2000,18.96,NaN,NaN,0.301058,NaN,NaN,NaN,21.262750,3.028780
94,2012-01-10,23.970,24.170,23.446,23.550,543583684,1.944,21.990,19.063,8.007,...,3.1900,16.32,NaN,NaN,0.737638,NaN,NaN,NaN,22.846231,3.115735
115,2012-01-11,21.365,21.536,21.220,21.305,361447688,1.950,21.210,17.206,11.099,...,3.5000,16.69,NaN,NaN,0.547356,NaN,NaN,NaN,21.805900,3.010867
31,2012-02-07,20.883,21.195,20.843,21.161,400091104,2.027,30.771,14.526,9.693,...,2.7341,16.80,NaN,NaN,0.464449,NaN,NaN,NaN,21.374190,3.003750


In [ ]:
T_df.tail()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,NATURALGAS,VIX,MA7,MA21,MACD,20SD,upper_band,lower_band,EMA,logmomentum
2432,2022-12-04,165.28,169.90,165.22,168.61,201458,332.21,214.97,129.00,345.97,...,6.778,24.71,160.641429,159.780476,-2.479818,11.070983,181.922442,137.638510,169.311917,5.121640
2451,2022-12-05,145.11,146.21,138.82,143.90,354955,248.08,194.21,113.88,177.98,...,7.789,30.51,159.127143,159.203810,-0.919288,11.495772,182.195354,136.212265,152.370639,4.962145
2490,2022-12-07,143.71,148.47,143.65,145.87,115834,232.17,163.25,114.77,174.45,...,6.144,27.76,156.328571,158.940000,0.661348,11.751236,182.442472,135.437528,148.036880,4.975837
2513,2022-12-08,169.39,172.18,168.73,172.10,83724,300.02,180.50,122.92,249.42,...,8.755,21.20,159.952857,159.945714,-0.090990,12.038046,184.021806,135.869622,164.078960,5.142248
2533,2022-12-09,158.21,164.29,158.21,164.20,111706,304.74,169.26,112.54,236.80,...,8.455,23.10,162.672857,160.382857,0.486306,12.037063,184.456982,136.308732,164.159653,5.094976


Drop first 21 rows (because moving average will be invalid)

In [ ]:
dataset = T_df.iloc[20:,:].reset_index(drop=True)

In [ ]:
dataset.head()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,NATURALGAS,VIX,MA7,MA21,MACD,20SD,upper_band,lower_band,EMA,logmomentum
0,2012-05-11,20.840,20.992,20.629,20.879,529284896,2.100,21.250,17.091,11.177,...,3.34,18.42,22.055429,21.933667,-0.053040,1.401371,24.736408,19.130925,21.663794,2.989664
1,2012-05-12,20.318,20.330,19.242,19.243,1044637412,2.247,27.710,17.212,11.910,...,3.40,16.46,21.402429,21.895952,-0.035627,1.532977,24.961907,18.829998,20.049931,2.903781
2,2012-05-17,19.475,19.554,18.933,18.933,717218852,1.905,38.000,15.591,10.281,...,2.61,24.49,21.169143,21.765524,0.069023,1.617847,25.001218,18.529830,19.305310,2.886643
3,2012-05-18,19.070,19.408,18.881,18.942,732287052,1.837,38.232,15.025,9.994,...,2.56,25.10,21.003571,21.546095,0.207817,1.728865,25.003825,18.088366,19.063103,2.887144
4,2012-05-21,19.089,20.055,19.073,20.046,631091160,1.918,34.030,15.368,10.249,...,2.59,22.01,20.755286,21.486143,0.406040,1.760042,25.006227,17.966059,19.718368,2.946857


In [ ]:
dataset.tail()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,NATURALGAS,VIX,MA7,MA21,MACD,20SD,upper_band,lower_band,EMA,logmomentum
2516,2022-12-04,165.28,169.90,165.22,168.61,201458,332.21,214.97,129.00,345.97,...,6.778,24.71,160.641429,159.780476,-2.479818,11.070983,181.922442,137.638510,169.311917,5.121640
2517,2022-12-05,145.11,146.21,138.82,143.90,354955,248.08,194.21,113.88,177.98,...,7.789,30.51,159.127143,159.203810,-0.919288,11.495772,182.195354,136.212265,152.370639,4.962145
2518,2022-12-07,143.71,148.47,143.65,145.87,115834,232.17,163.25,114.77,174.45,...,6.144,27.76,156.328571,158.940000,0.661348,11.751236,182.442472,135.437528,148.036880,4.975837
2519,2022-12-08,169.39,172.18,168.73,172.10,83724,300.02,180.50,122.92,249.42,...,8.755,21.20,159.952857,159.945714,-0.090990,12.038046,184.021806,135.869622,164.078960,5.142248
2520,2022-12-09,158.21,164.29,158.21,164.20,111706,304.74,169.26,112.54,236.80,...,8.455,23.10,162.672857,160.382857,0.486306,12.037063,184.456982,136.308732,164.159653,5.094976


Get Fourier transform features

In [ ]:
dataset_F = get_fourier_transfer(dataset, FOURIER_COMPONENTS)
Final_data = pd.concat([dataset, dataset_F], axis=1)

In [ ]:
Final_data.head()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,absolute of 6 comp,angle of 6 comp,absolute of 9 comp,angle of 9 comp,absolute of 10 comp,angle of 10 comp,absolute of 11 comp,angle of 11 comp,absolute of 12 comp,angle of 12 comp
0,2012-05-11,20.840,20.992,20.629,20.879,529284896,2.100,21.250,17.091,11.177,...,90.468261,-0.045942,89.513648,-0.024799,88.384619,-0.020354,87.926361,-0.013732,88.222580,-0.016938
1,2012-05-12,20.318,20.330,19.242,19.243,1044637412,2.247,27.710,17.212,11.910,...,89.797052,-0.046256,88.518210,-0.024901,87.294944,-0.020473,86.759019,-0.013912,86.977544,-0.017273
2,2012-05-17,19.475,19.554,18.933,18.933,717218852,1.905,38.000,15.591,10.281,...,89.125587,-0.046565,87.523060,-0.024993,86.206234,-0.020583,85.592984,-0.014085,85.733613,-0.017602
3,2012-05-18,19.070,19.408,18.881,18.942,732287052,1.837,38.232,15.025,9.994,...,88.453919,-0.046868,86.528367,-0.025075,85.118706,-0.020682,84.428527,-0.014253,84.491126,-0.017924
4,2012-05-21,19.089,20.055,19.073,20.046,631091160,1.918,34.030,15.368,10.249,...,87.782101,-0.047165,85.534298,-0.025145,84.032580,-0.020770,83.265918,-0.014414,83.250416,-0.018239


In [ ]:
Final_data.tail()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,absolute of 6 comp,angle of 6 comp,absolute of 9 comp,angle of 9 comp,absolute of 10 comp,angle of 10 comp,absolute of 11 comp,angle of 11 comp,absolute of 12 comp,angle of 12 comp
2516,2022-12-04,165.28,169.90,165.22,168.61,201458,332.21,214.97,129.00,345.97,...,93.818640,-0.044292,94.489323,-0.024140,93.839728,-0.019616,93.773106,-0.012755,94.452565,-0.015184
2517,2022-12-05,145.11,146.21,138.82,143.90,354955,248.08,194.21,113.88,177.98,...,93.149495,-0.044632,93.494949,-0.024291,92.748545,-0.019781,92.603332,-0.012960,93.207053,-0.015544
2518,2022-12-07,143.71,148.47,143.65,145.87,115834,232.17,163.25,114.77,174.45,...,92.479832,-0.044967,92.500027,-0.024432,91.657220,-0.019937,91.433496,-0.013160,91.960961,-0.015900
2519,2022-12-08,169.39,172.18,168.73,172.10,83724,300.02,180.50,122.92,249.42,...,91.809704,-0.045297,91.504725,-0.024564,90.565977,-0.020085,90.263873,-0.013356,90.714626,-0.016251
2520,2022-12-09,158.21,164.29,158.21,164.20,111706,304.74,169.26,112.54,236.80,...,91.139163,-0.045622,90.509209,-0.024686,89.475036,-0.020224,89.094736,-0.013547,89.468387,-0.016597


In [ ]:
Final_data['Date']

0      2012-05-11
1      2012-05-12
2      2012-05-17
3      2012-05-18
4      2012-05-21
          ...    
2516   2022-12-04
2517   2022-12-05
2518   2022-12-07
2519   2022-12-08
2520   2022-12-09
Name: Date, Length: 2521, dtype: datetime64[ns]

Include interest timeframe only

In [ ]:
date_from = datetime.strptime(DATE_FROM, DATE_FORMAT)
date_to = datetime.strptime(DATE_TO, DATE_FORMAT)

#date_from_backward = date_from - timedelta(days=BACKWARD_LOOKING_NUM)
#df = Final_data[[i >= date_from_backward and i <= date_to for i in Final_data['Date']]]
#df = df.sort_values(by="Date")

In [ ]:
index_start = Final_data[Final_data['Date'] == date_from].index[0]
index_end = Final_data[Final_data['Date'] == date_to].index[0]

index_start_backward = index_start - BACKWARD_LOOKING_NUM

In [ ]:
df = Final_data[index_start_backward:index_end + 1]

In [ ]:
df.head(BACKWARD_LOOKING_NUM + 1)

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,absolute of 6 comp,angle of 6 comp,absolute of 9 comp,angle of 9 comp,absolute of 10 comp,angle of 10 comp,absolute of 11 comp,angle of 11 comp,absolute of 12 comp,angle of 12 comp
2419,2022-05-16,145.69,147.55,144.22,146.00,202988,241.64,200.08,114.95,187.51,...,148.705847,-0.002249,165.626337,0.005753,169.127271,0.007286,170.919976,0.006648,171.176549,0.008848
2420,2022-05-17,147.74,149.80,146.71,149.25,169335,254.85,202.51,116.90,190.82,...,148.305155,-0.002672,165.309313,0.005476,168.861383,0.007091,170.715530,0.006588,171.048568,0.008878
2421,2022-05-18,148.60,148.81,140.00,140.29,137609,234.30,190.76,111.75,176.02,...,147.900008,-0.003098,164.982044,0.005195,168.583289,0.006891,170.497662,0.006523,170.906926,0.008900
2422,2022-05-19,139.50,141.68,136.63,138.41,278027,239.62,192.91,111.34,185.12,...,147.490430,-0.003525,164.644519,0.004911,168.292948,0.006687,170.266291,0.006454,170.751477,0.008915
2423,2022-05-20,139.35,140.51,132.64,137.33,239375,220.78,193.50,109.35,186.39,...,147.076440,-0.003954,164.296731,0.004624,167.990327,0.006478,170.021341,0.006380,170.582083,0.008923
2424,2022-05-23,138.69,143.47,137.58,141.21,144881,219.07,182.32,108.12,186.64,...,146.658061,-0.004385,163.938674,0.004333,167.675393,0.006265,169.762741,0.006302,170.398607,0.008924
2425,2022-05-24,140.56,141.97,137.37,141.61,149627,211.50,182.70,106.67,181.92,...,146.235316,-0.004817,163.570345,0.004039,167.348120,0.006048,169.490423,0.006219,170.200921,0.008918
2426,2022-05-25,140.21,141.81,138.17,140.19,143465,218.58,182.83,105.77,189.13,...,145.808228,-0.005250,163.191744,0.003742,167.008482,0.005826,169.204327,0.006132,169.988900,0.008904
2427,2022-05-26,139.84,144.37,137.17,144.21,94866,237.69,191.58,108.51,192.50,...,145.376821,-0.005686,162.802874,0.003441,166.656461,0.005601,168.904394,0.006040,169.762424,0.008883
2428,2022-05-27,143.41,149.98,143.36,149.96,76680,253.36,195.40,113.02,195.10,...,144.941118,-0.006123,162.403739,0.003138,166.292040,0.005372,168.590573,0.005944,169.521379,0.008855


In [ ]:
df.tail()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,absolute of 6 comp,angle of 6 comp,absolute of 9 comp,angle of 9 comp,absolute of 10 comp,angle of 10 comp,absolute of 11 comp,angle of 11 comp,absolute of 12 comp,angle of 12 comp
2497,2022-09-15,155.77,155.77,151.17,151.41,140943,300.32,148.11,103.55,232.54,...,106.364824,-0.037063,113.056810,-0.019901,114.251647,-0.015222,115.623212,-0.008261,117.563938,-0.007943
2498,2022-09-16,151.13,151.58,148.39,150.98,159521,303.40,146.60,104.25,240.69,...,105.715373,-0.037473,112.104368,-0.020178,113.204992,-0.015499,114.506404,-0.008515,116.392331,-0.008328
2499,2022-09-19,149.60,155.02,148.73,155.02,174953,309.53,148.52,104.82,244.50,...,105.064425,-0.037882,111.148254,-0.020450,112.154042,-0.015773,113.384403,-0.008768,115.213955,-0.008714
2500,2022-09-20,153.88,158.11,153.13,156.51,193568,307.51,146.07,102.38,243.30,...,104.412031,-0.038287,110.188628,-0.020716,111.099010,-0.016042,112.257469,-0.009020,114.029117,-0.009101
2501,2022-09-21,156.48,158.73,152.25,152.49,296918,299.33,140.86,99.71,234.94,...,103.758240,-0.038689,109.225649,-0.020978,110.040108,-0.016307,111.125864,-0.009271,112.838126,-0.009488


Check NA and fill them

In [ ]:
dataset = df
dataset.isnull().sum()
dataset.iloc[:, 1:] = pd.concat([dataset.iloc[:, 1:].ffill(), dataset.iloc[:, 1:].bfill()]).groupby(level=0).mean()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [ ]:
dataset.tail()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,absolute of 6 comp,angle of 6 comp,absolute of 9 comp,angle of 9 comp,absolute of 10 comp,angle of 10 comp,absolute of 11 comp,angle of 11 comp,absolute of 12 comp,angle of 12 comp
2497,2022-09-15,155.77,155.77,151.17,151.41,140943.0,300.32,148.11,103.55,232.54,...,106.364824,-0.037063,113.056810,-0.019901,114.251647,-0.015222,115.623212,-0.008261,117.563938,-0.007943
2498,2022-09-16,151.13,151.58,148.39,150.98,159521.0,303.40,146.60,104.25,240.69,...,105.715373,-0.037473,112.104368,-0.020178,113.204992,-0.015499,114.506404,-0.008515,116.392331,-0.008328
2499,2022-09-19,149.60,155.02,148.73,155.02,174953.0,309.53,148.52,104.82,244.50,...,105.064425,-0.037882,111.148254,-0.020450,112.154042,-0.015773,113.384403,-0.008768,115.213955,-0.008714
2500,2022-09-20,153.88,158.11,153.13,156.51,193568.0,307.51,146.07,102.38,243.30,...,104.412031,-0.038287,110.188628,-0.020716,111.099010,-0.016042,112.257469,-0.009020,114.029117,-0.009101
2501,2022-09-21,156.48,158.73,152.25,152.49,296918.0,299.33,140.86,99.71,234.94,...,103.758240,-0.038689,109.225649,-0.020978,110.040108,-0.016307,111.125864,-0.009271,112.838126,-0.009488


In [ ]:
datetime_series = pd.to_datetime(dataset['Date'])
datetime_index = pd.DatetimeIndex(datetime_series.values)
dataset = dataset.set_index(datetime_index)
dataset = dataset.sort_values(by='Date')
dataset = dataset.drop(columns='Date')

In [ ]:
dataset.tail()

,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,AMZN,...,absolute of 6 comp,angle of 6 comp,absolute of 9 comp,angle of 9 comp,absolute of 10 comp,angle of 10 comp,absolute of 11 comp,angle of 11 comp,absolute of 12 comp,angle of 12 comp
2022-09-15,155.77,155.77,151.17,151.41,140943.0,300.32,148.11,103.55,232.54,123.93,...,106.364824,-0.037063,113.056810,-0.019901,114.251647,-0.015222,115.623212,-0.008261,117.563938,-0.007943
2022-09-16,151.13,151.58,148.39,150.98,159521.0,303.40,146.60,104.25,240.69,123.85,...,105.715373,-0.037473,112.104368,-0.020178,113.204992,-0.015499,114.506404,-0.008515,116.392331,-0.008328
2022-09-19,149.60,155.02,148.73,155.02,174953.0,309.53,148.52,104.82,244.50,125.49,...,105.064425,-0.037882,111.148254,-0.020450,112.154042,-0.015773,113.384403,-0.008768,115.213955,-0.008714
2022-09-20,153.88,158.11,153.13,156.51,193568.0,307.51,146.07,102.38,243.30,122.18,...,104.412031,-0.038287,110.188628,-0.020716,111.099010,-0.016042,112.257469,-0.009020,114.029117,-0.009101
2022-09-21,156.48,158.73,152.25,152.49,296918.0,299.33,140.86,99.71,234.94,117.35,...,103.758240,-0.038689,109.225649,-0.020978,110.040108,-0.016307,111.125864,-0.009271,112.838126,-0.009488


# Trading Strategy

Check trend\
\
-1 = Downward trend\
 0 = Flat trend\
 1 = Upward trend 

In [ ]:
def GetPriceTrend(price_list):
  diff = price_list[-1] - price_list[0]

  #print(diff)
  if (abs(diff) / price_list[0]) <= TREND_FLAT_THREHOLD:
    return TREND_FLAT
  elif diff > 0:
    if diff > FEE_TRADING:
      return TREND_UPWARD
    return TREND_FLAT

  return TREND_DOWNWARD

In [ ]:
TREND_DOWNWARD = -1
TREND_FLAT = 0
TREND_UPWARD = 1

In [ ]:
# Load scaler/ index
X_scaler = load(open(X_SCALE_PKL, 'rb'))
y_scaler = load(open(Y_SCALE_PKL, 'rb'))

Normalise Data

In [ ]:
X_scale_df = X_scaler.transform(dataset)

In [ ]:
X_scale_df

array([[ 0.56829483,  0.5753181 ,  0.57742903, ..., -0.36208501,
         0.91960866, -0.32487241],
       [ 0.59268159,  0.60191243,  0.60753494, ..., -0.36256722,
         0.91809011, -0.3246482 ],
       [ 0.60291214,  0.59021092,  0.52640615, ..., -0.36308625,
         0.91640946, -0.32447828],
       ...,
       [ 0.61480812,  0.66361127,  0.63195821, ..., -0.48577335,
         0.25558876, -0.45746079],
       [ 0.66572292,  0.70013415,  0.68515742, ..., -0.48779517,
         0.24153016, -0.46038067],
       [ 0.69665247,  0.70746237,  0.67451758, ..., -0.48980619,
         0.22739854, -0.46330368]])

In [ ]:
G_model = tf.keras.models.load_model(MODEL_FILE)

 Loop through all trading date

In [ ]:
hold_position = 0
hold_capital = CAPITAL

for i in range(BACKWARD_LOOKING_NUM, len(X_scale_df - 1)):
    X_value = np.expand_dims(X_scale_df[i - BACKWARD_LOOKING_NUM:i], axis=0)
    y_predicted = G_model(X_value)
    rescaled_predicted_y = y_scaler.inverse_transform(y_predicted)
    rescaled_actual_y = dataset[i:i+1]['Close'].tolist()
    #print(str(i) + ": " + str(rescaled_predicted_y))
    #print(str(i) + ": " + str(rescaled_actual_y))

    trend = GetPriceTrend(rescaled_predicted_y[0])

    if hold_position > 0:
      if trend == TREND_DOWNWARD:
        gain = hold_position * rescaled_actual_y[0]
        print("SOLD: " + str(hold_position) + " | gain: " + str(gain))
        hold_capital += gain
        hold_position = 0
      else:
        print("HOLD: " + str(hold_position) + " | gain: " + str(-COST_HOLD))
        hold_capital -= COST_HOLD
    else:
      if trend == TREND_UPWARD:
        hold_capital -= COST_BUY
        hold_position = math.floor(hold_capital / rescaled_actual_y[0])
        cost = hold_position * rescaled_actual_y[0]
        hold_capital -= cost
        print("BUY: " + str(hold_position) + " | gain: " + str(-(cost + COST_BUY)))
      else:
        print("DO NOTHING")
    #print("------------------------------------")

if hold_position > 0:
  gain = hold_position * dataset[-1:]['Close'].tolist()[0]
  print("SOLD: " + str(hold_position) + " | gain: " + str(gain))
  hold_capital += gain
  hold_position = 0

BUY: 58 | gain: -9983.900000000001
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0.2
HOLD: 58 | gain: -0

In [ ]:
hold_capital

8850.119999999999